In [1]:
# Bokeh libraries
import pandas as pd
from bokeh.io import output_notebook
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, NumeralTickFormatter, HoverTool

# Import data
from read_nba_data import player_stats

# Set output stream
output_notebook()

Loading BokehJS ...

In [2]:
# Find players who took at least 1 three-point shot during the season
three_takers = player_stats[player_stats['play3PA'] > 0].copy()

# Make a Full Name column
three_takers.loc[:, 'name'] = three_takers['playFNm'].astype(str) + ' ' + \
                              three_takers['playLNm'].astype(str)

# Aggregate three-point player attempts (play3PA) and three-point player made (play3PM) for each player
datetime_columns = three_takers.select_dtypes(include=['datetime64']).columns
three_takers = (three_takers.drop(columns=datetime_columns)
                            .groupby('name')
                            .sum()
                            .loc[:, ['play3PA', 'play3PM']]
                            .sort_values('play3PA', ascending=False))

# Filter out anyone who didn't take at least 100 three-point shots
three_takers = three_takers[three_takers['play3PA'] >= 200].reset_index()

# Add a column with a calculated three-point percentage (made/attempted)
three_takers['pct3PM'] = three_takers['play3PM'] / three_takers['play3PA']

In [3]:
# Store the data in a ColumnDataSource
three_takers_cds = ColumnDataSource(three_takers)

#Specify the selection tools to be made available
select_tools = ['box_select', 'lasso_select', 'poly_select', 'tap', 'reset']

# Create the figure
fig = figure(height=400,
             width=600,
             x_axis_label='Attempts',
             y_axis_label='Percentage Made',
             title='3-Point-Shot Attempted vs. Percentage Made (min. 100 3PA), 2017-18',
             toolbar_location='below',
             tools=select_tools)

# Format the y-axis tick label as percentages
fig.yaxis[0].formatter = NumeralTickFormatter(format='00.0%')

# Add square representing each player
fig.square(x='play3PA',
           y='pct3PM',
           source=three_takers_cds,
           color='blue',
           selection_color='deepskyblue',
           nonselection_color='lightgray',
           nonselection_alpha=0.3)

# Visualize
show(fig)

In [4]:
tooltips = [('Player', '@name'), 
            ('Three-Pointers Attempted','@play3PA'),
            ('Three-Pointers Masde','@play3PM'),
            ('Three-Point Percentage','@play3PT'),
            ]

hover_glyph = fig.circle(x='play3PA',y='pct3PM', source=three_takers,
                         size=15, alpha=0,
                         hover_fill_color='black',hover_alpha=0.5)

fig.add_tools(HoverTool(tooltips=tooltips, renderers=[hover_glyph]))

# Visualize
show(fig)